<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto_Part 1</font></h1>

## Problem part 1


In [27]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

###### Scrape the List of postal codes of Canada

In [3]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(wiki_url).text

In [4]:
soup = BeautifulSoup(source, 'xml')

In [8]:
table=soup.find('table')

In [9]:
c_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = c_names)

In [15]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    #print(tr_cell)
    for td_cell in tr_cell.find_all('td'):
        td_data=[]
        td_data.append(td_cell.find('b').text.strip())
        td_data.append('Not assigned')
        td_data.append('Not assigned')
        td_data_idx = 1
        for td_ele in td_cell.find_all('a'):
            td_data[td_data_idx] = td_ele.text.strip()
            td_data_idx += 1
            if td_data_idx > 2:
                break
        #print(td_data)
        df.loc[len(df)] = td_data
        #df

###### Data Cleaning
remove rows where Borough is 'Not assigned'

In [26]:
df=df[df['Borough']!='Not assigned']
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned


In [31]:
for index, row in df.iterrows():
    if row[2]=='Not assigned':
        row[2] = row[1]
    
#df[df[]=='Not assigned']=df['Borough']
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park


In [32]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [33]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [34]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [35]:
df_merge.drop_duplicates(inplace=True)

In [36]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [37]:
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,Lawrence Manor
4,M7A,Queen's Park,Queen's Park
